# Костин Сергей, ИУ5-65Б
## Рубежный Контроль №1, В9

Задача №2.

Для заданного набора данных проведите обработку пропусков в данных для одного категориального и одного количественного признака. Какие способы обработки пропусков в данных для категориальных и количественных признаков Вы использовали? Какие признаки Вы будете использовать для дальнейшего построения моделей машинного обучения и почему?

# Импорты и либы

In [91]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import MissingIndicator

# Функции

In [92]:
def descr(df):
    print("Размер датасета:", df.shape)
    print("Столбцы и типы данных:\n", df.dtypes)
    print("Количество пропусков:\n", df.isnull().sum())

In [93]:
def normal_total(mean, std, total):
    percentage = np.random.normal(mean, std)
    percentage = np.clip(percentage, 0, 100)
    missing_count = int(total * (percentage / 100))
    return missing_count

# Код

## Загрузка/Подготовка данных

In [94]:
iris = load_iris(as_frame=True)
df = iris.frame

In [95]:
descr(df)

Размер датасета: (150, 5)
Столбцы и типы данных:
 sepal length (cm)    float64
sepal width (cm)     float64
petal length (cm)    float64
petal width (cm)     float64
target                 int32
dtype: object
Количество пропусков:
 sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
target               0
dtype: int64


### Искусственные пропуски
В исходном ДС нету пропусков, создадим их искусственно

In [96]:
# Сколько столбцов должны быть случайно отброшены
damage = 1

# Сколько строк % должно быть случайно отброшены (нормальное распределение)
m = 30  # среднее значение процента пропусков
u = 15   # стандартное отклонение процента пропусков

In [97]:
df_dmg = df.copy()
np.random.seed(42)
# .difference(['target'])
random_columns = np.random.choice(df_dmg.columns, size=damage, replace=False).tolist() + ['target']

for column in random_columns:
    missing_count = normal_total(m, u, df_dmg.shape[0])
    missing_indices = np.random.choice(df_dmg.index, size=missing_count, replace=False)
    df_dmg.loc[missing_indices, column] = np.nan

In [98]:
descr(df_dmg)

Размер датасета: (150, 5)
Столбцы и типы данных:
 sepal length (cm)    float64
sepal width (cm)     float64
petal length (cm)    float64
petal width (cm)     float64
target               float64
dtype: object
Количество пропусков:
 sepal length (cm)     0
sepal width (cm)     55
petal length (cm)     0
petal width (cm)      0
target               75
dtype: int64


# Обработка пропусков

In [110]:
quantitative_columns = df.select_dtypes(include=['float64']).columns.tolist()
print(quantitative_columns)

categorical_columns = [ 'target' ] # df.select_dtypes(include=['object']).columns.tolist()
print(categorical_columns)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['target']


In [111]:
for col in quantitative_columns:
    imp_num = SimpleImputer(strategy='median')
    df_dmg[col] = imp_num.fit_transform(df_dmg[[col]])

for col in categorical_columns:
    imp_cat = SimpleImputer(strategy='most_frequent')
    df_dmg[col] = imp_cat.fit_transform(df_dmg[[col]])

In [112]:
descr(df_dmg)

Размер датасета: (150, 5)
Столбцы и типы данных:
 sepal length (cm)    float64
sepal width (cm)     float64
petal length (cm)    float64
petal width (cm)     float64
target               float64
dtype: object
Количество пропусков:
 sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
target               0
dtype: int64


# Сравнение

In [113]:
comparison = []

for column in random_columns:
    if df[column].dtype == 'float64':  # числовые данные
        metrics = {
            'column': column,
            'original_mean': df[column].mean(),
            'restored_mean': df_dmg[column].mean(),
            'original_median': df[column].median(),
            'restored_median': df_dmg[column].median(),
        }
        comparison.append(metrics)
    else:  # категориальные данные
        original_counts = df[column].value_counts().to_dict()
        restored_counts = df_dmg[column].value_counts().to_dict()
        for category in set(original_counts.keys()).union(set(restored_counts.keys())):
            comparison.append({
                'column': column,
                'category': category,
                'original_count': original_counts.get(category, 0),
                'restored_count': restored_counts.get(category, 0)
            })

comparison = pd.DataFrame(comparison)
print(comparison)
comparison.head()

             column  original_mean  restored_mean  original_median  \
0  sepal width (cm)       3.057333       3.042667              3.0   
1            target            NaN            NaN              NaN   
2            target            NaN            NaN              NaN   
3            target            NaN            NaN              NaN   

   restored_median  category  original_count  restored_count  
0              3.0       NaN             NaN             NaN  
1              NaN       0.0            50.0            18.0  
2              NaN       1.0            50.0            24.0  
3              NaN       2.0            50.0           108.0  


,column,original_mean,restored_mean,original_median,restored_median,category,original_count,restored_count
0,sepal width (cm),3.057333,3.042667,3.0,3.0,NaN,NaN,NaN
1,target,NaN,NaN,NaN,NaN,0.0,50.0,18.0
2,target,NaN,NaN,NaN,NaN,1.0,50.0,24.0
3,target,NaN,NaN,NaN,NaN,2.0,50.0,108.0


# Выводы

### Какие способы обработки пропусков в данных для категориальных и количественных признаков Вы использовали?

Для обработки пропусков использовались импьютеры (SimpleImputer)

Для категориальных - со стратегией most_frequent
Для количественных - со стратегией median

### Какие признаки Вы будете использовать для дальнейшего построения моделей машинного обучения и почему?

\* Датасет создавался "неполным" искусственно

В случае, если бы такой датасет был был получен от заказчика - заказчику было бы отказано, тк единственный категориальный признак - целевой и он имеет много пропусков.

Импьютация количественно столбца, в целом, не привела к большим отклонениям, поэтому было бы целесообразно посмотреть на реальное поведение модели и делать выводы уже после этого.